In [558]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tqdm import tqdm, tqdm_notebook
from itertools import product

import warnings
warnings.filterwarnings('ignore')

In [595]:
def round_thresh(x, t):
    if x % 1 > t:
        return np.ceil(x)
    else:
        return np.floor(x)

def rsme(y1, y2):
    return np.sqrt(mean_squared_error(y1, y2))

In [559]:
vendas = pd.read_csv('../../../data/interim/Vendas.csv', encoding = "ISO-8859-1")

In [560]:
vendas = vendas.replace(['ST', 'CORSÁRIO SUPLEX', 'ESP', 'ESPECIAL'], ['00','CORSARIO SUPLEX', 'GG', 'GG'])

In [561]:
vendas['DATE_BLOCK'] = LabelEncoder().fit_transform(vendas.ANO.apply(str) + vendas.MES.apply(lambda x: str(x).zfill(5)))

### Fazer as vendas mensais 

In [562]:
a = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO', 'DATE_BLOCK'])

In [563]:
df = a.sum().reset_index().drop(['DIA', 'VLR_UNIT'], 1)
df = df.sort_values(by=['ANO','MES', 'DESCRICAO', 'TAMANHO', 'NOME'])

In [564]:
df[df.ANO == 2018].head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,DATE_BLOCK,QUANTIDADE
4399,2018.0,1.0,BERMUDA CICLISTA COTTON,ALDEIA DA SERRA,02,24,1.0
4402,2018.0,1.0,BERMUDA CICLISTA COTTON,VERBO DIVINO,02,24,3.0
4396,2018.0,1.0,BERMUDA CICLISTA COTTON,ACLIMAÇÃO,04,24,8.0
4400,2018.0,1.0,BERMUDA CICLISTA COTTON,ALDEIA DA SERRA,04,24,1.0
4403,2018.0,1.0,BERMUDA CICLISTA COTTON,VERBO DIVINO,04,24,13.0


In [565]:
df_piv = df.pivot_table(index=['DESCRICAO', 'NOME', 'TAMANHO'], columns=['DATE_BLOCK'], 
                            values='QUANTIDADE',aggfunc='sum', dropna=True).fillna(0).reset_index()

In [596]:
df_piv.head()

DATE_BLOCK,DESCRICAO,NOME,TAMANHO,0,1,2,3,4,5,6,...,19,20,21,22,23,24,25,26,27,28
0,BERMUDA CICLISTA COLEGIAL,ACLIMAÇÃO,G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BERMUDA CICLISTA COLEGIAL,ACLIMAÇÃO,PP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,P,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,PP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BERMUDA CICLISTA COLEGIAL,ITAIM,M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [569]:
a = dict(enumerate(product([2016, 2017, 2018], [i for i in range(1,13)])))

In [571]:
def migue(x):
    if type(x) == str:
        return np.nan
    else:
        return x

In [804]:
final_df = 0
for i in tqdm(range(len(df_piv))):
    k = pd.DataFrame({'QUANTIDADE' : df_piv.iloc[i].values[3:],
                      'LAST_MONTH' : df_piv.iloc[i].shift(1).apply(lambda x: migue(x)).values[3:],
                      'LAST_YEAR' : df_piv.iloc[i].shift(12).apply(lambda x: migue(x)).values[3:],
                     'LAST_YEAR_2' : df_piv.iloc[i].shift(24).apply(lambda x: migue(x)).values[3:]})
    ano = list(map(lambda x: a[x][0], pd.DataFrame(df_piv.iloc[i].values[3:]).index))
    mes = list(map(lambda x: a[x][1], pd.DataFrame(df_piv.iloc[i].values[3:]).index))
    k['DESCRICAO'] = df_piv.iloc[i].DESCRICAO
    k['TAMANHO'] = df_piv.iloc[i].TAMANHO
    k['NOME'] = df_piv.iloc[i].NOME
    k['ANO'] = ano
    k['MES'] = mes
    
    try:
        final_df = final_df.append(k)
    except:
        final_df = k
        
final_df['LAST_YEAR_MEAN'] = (final_df['LAST_YEAR'] + final_df['LAST_YEAR_2']) / 2
final_df['DATE_BLOCK'] = final_df.index
final_df = final_df[['DATE_BLOCK', 'ANO', 'MES', 'DESCRICAO', 'TAMANHO', 'NOME', 'LAST_MONTH', 'LAST_YEAR', 'LAST_YEAR_2','LAST_YEAR_MEAN', 'QUANTIDADE']]

100%|██████████| 670/670 [00:03<00:00, 168.26it/s]


In [805]:
final_df = final_df.sort_values(by=['ANO', 'MES', 'DESCRICAO', 'TAMANHO', 'NOME'])

#### Ultimo ano

Do ultimo ano, não temos registrado Itaim, nem as outras 2 lojas até o mes 3

In [806]:
final_df.index = np.arange(len(final_df))

In [807]:
final_df = final_df.drop(final_df[final_df.ANO == 2018][final_df.MES > 3][final_df.NOME != 'VERBO DIVINO'].index)

In [808]:
final_df = final_df.drop(final_df[final_df.ANO == 2018][final_df.NOME == 'ITAIM'].index)

In [809]:
treino = final_df[final_df.ANO < 2018]

In [810]:
validacao = final_df[final_df.ANO == 2018]

#### Data

In [811]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor

In [812]:
le_DESC = LabelEncoder()
treino['DESCRICAO'] = le_DESC.fit_transform(treino.DESCRICAO)
validacao['DESCRICAO'] = le_DESC.transform(validacao.DESCRICAO)

In [813]:
le_TAM = LabelEncoder()
treino['TAMANHO'] = le_TAM.fit_transform(treino.TAMANHO)
validacao['TAMANHO'] = le_TAM.transform(validacao.TAMANHO)

In [814]:
le_NOME = LabelEncoder()
treino['NOME'] = le_NOME.fit_transform(treino.NOME)
validacao['NOME'] = le_NOME.transform(validacao.NOME)

In [818]:
X_train = treino.drop(['DATE_BLOCK','LAST_YEAR', 'LAST_MONTH', 'LAST_YEAR_2', 'LAST_YEAR_MEAN'], 1).iloc[:, :-1]
y_train = treino.drop(['DATE_BLOCK','LAST_YEAR', 'LAST_MONTH', 'LAST_YEAR_2', 'LAST_YEAR_MEAN'], 1).iloc[:, -1]

X_test = validacao.drop(['DATE_BLOCK','LAST_YEAR', 'LAST_MONTH', 'LAST_YEAR_2', 'LAST_YEAR_MEAN'], 1).iloc[:, :-1]
y_test = validacao.drop(['DATE_BLOCK','LAST_YEAR', 'LAST_MONTH', 'LAST_YEAR_2', 'LAST_YEAR_MEAN'], 1).iloc[:, -1]

## Treino - Modelo 1

#### Apenas dados normais

In [819]:
import xgboost as xgb

param = {'max_depth': 7, 
         'subsample': .85, 
         'colsample_bylevel': 1,
         'n_estimators': 200, 
         'learning_rate': .05,
         'seed':0,  # 1
         'silent':0,
         'reg_alpha':.1,
         'eval_metric':'rmse',
        }

clf_xgb = xgb.XGBRegressor(**param)

In [820]:
clf_xgb.fit(X_train.values, y_train.values, eval_set=[(X_test.values, y_test.values)], early_stopping_rounds=10)

[0]	validation_0-rmse:10.6734
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:10.2667
[2]	validation_0-rmse:9.87834
[3]	validation_0-rmse:9.51925
[4]	validation_0-rmse:9.17357
[5]	validation_0-rmse:8.85848
[6]	validation_0-rmse:8.55381
[7]	validation_0-rmse:8.24409
[8]	validation_0-rmse:7.93802
[9]	validation_0-rmse:7.65531
[10]	validation_0-rmse:7.37155
[11]	validation_0-rmse:7.12929
[12]	validation_0-rmse:6.90894
[13]	validation_0-rmse:6.69393
[14]	validation_0-rmse:6.47836
[15]	validation_0-rmse:6.31327
[16]	validation_0-rmse:6.14079
[17]	validation_0-rmse:5.96225
[18]	validation_0-rmse:5.80478
[19]	validation_0-rmse:5.66795
[20]	validation_0-rmse:5.54622
[21]	validation_0-rmse:5.41848
[22]	validation_0-rmse:5.31191
[23]	validation_0-rmse:5.21023
[24]	validation_0-rmse:5.12291
[25]	validation_0-rmse:5.01374
[26]	validation_0-rmse:4.92715
[27]	validation_0-rmse:4.85522
[28]	validation_0-rmse:4.77006
[29]	validation_0-rmse:4.69621
[30]	validation

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='rmse', gamma=0, learning_rate=0.05,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=0, subsample=0.85)

In [821]:
y_pred = clf_xgb.predict(X_test.values)

In [822]:
rsme(y_test, np.round(y_pred))

4.257547266637811

In [823]:
list(zip(y_test, y_pred))

[(0.0, 0.561448),
 (0.0, 0.46445584),
 (0.0, 2.0748923),
 (0.0, 0.4013201),
 (0.0, 3.514013),
 (0.0, 0.809558),
 (0.0, 0.65000165),
 (0.0, 3.734437),
 (0.0, 2.2991052),
 (0.0, 2.7903144),
 (0.0, 4.640706),
 (1.0, 1.5404485),
 (3.0, 4.753983),
 (8.0, 4.7068253),
 (1.0, 2.7222457),
 (13.0, 19.132698),
 (12.0, 5.34741),
 (2.0, 2.6877677),
 (20.0, 16.976475),
 (8.0, 5.2106),
 (0.0, 2.7774327),
 (13.0, 20.104206),
 (12.0, 6.281549),
 (0.0, 3.7412527),
 (8.0, 27.887262),
 (5.0, 2.8472002),
 (6.0, 1.9266114),
 (16.0, 16.31211),
 (0.0, 0.68142223),
 (1.0, 2.7550688),
 (0.0, 0.5777123),
 (0.0, 0.71140707),
 (0.0, 0.6220554),
 (4.0, 4.9538336),
 (1.0, 0.8185476),
 (0.0, 0.755996),
 (2.0, 8.809428),
 (3.0, 1.6189026),
 (0.0, 1.509479),
 (8.0, 8.966591),
 (5.0, 6.4005146),
 (3.0, 2.6221113),
 (11.0, 19.004574),
 (3.0, 7.1050234),
 (2.0, 2.949945),
 (27.0, 21.005943),
 (9.0, 7.0843334),
 (1.0, 4.4280515),
 (37.0, 44.271305),
 (9.0, 7.6660986),
 (4.0, 4.3347526),
 (26.0, 34.416653),
 (6.0, 7.516517)

In [824]:
rsme(y_test, (y_last_year_mean + y_pred) / 2)

4.255673322815765

## Treino - Modelo 2

#### Apenas dados normais + new_features

In [825]:
X_train = treino.drop(['LAST_YEAR_2', 'LAST_YEAR_MEAN'], 1).iloc[:, :-1]
y_train = treino.iloc[:, -1]

X_test = validacao.drop(['LAST_YEAR_2', 'LAST_YEAR'], 1).iloc[:, :-1]
y_test = validacao.iloc[:, -1]

In [826]:
param = {'max_depth': 5, 
         'subsample': .7, 
         'colsample_bylevel': .7,
         'n_estimators': 200, 
         'learning_rate': .05,
         'random_state':0,  # 1
         'silent':2,
         'reg_alpha': .4,
         'eval_metric':'rmse',
        'missing': np.nan
        }

clf_xgb = xgb.XGBRegressor(**param)

In [827]:
clf_xgb.fit(X_train.values, y_train.values, eval_set=[(X_test.values, y_test.values)], early_stopping_rounds=10, verbose=0)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
       colsample_bytree=1, eval_metric='rmse', gamma=0, learning_rate=0.05,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.4, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=2, subsample=0.7)

In [828]:
y_pred_2 = clf_xgb.predict(X_test.values)

In [829]:
rsme(y_test, (y_pred_2 + y_pred + y_last_year_mean)/3)

4.117454175976899

### Validação

In [498]:
y_last_year = final_df[final_df.ANO == 2018]['LAST_YEAR']

In [502]:
rsme(y_test, y_last_year)

5.6950639981669795

In [545]:
y_last_year_2 = final_df[final_df.ANO == 2018]['LAST_YEAR_2']

In [546]:
rsme(y_test, y_last_year_2)

6.140015447460847

In [548]:
y_last_year_mean = final_df[final_df.ANO == 2018]['LAST_YEAR_MEAN']

In [550]:
rsme(y_test, np.round(y_last_year_mean))

4.789359042926411

In [551]:
y_last_month = final_df[final_df.ANO == 2018]['LAST_MONTH']

In [552]:
rsme(y_test, y_last_month)

11.100297928003155

In [2411]:
mean_squared_error(y_val, list(map(up, y_lin)))

232.60609137055837

In [2412]:
mean_squared_error(validacao.QUANTIDADE.values, (y_rf + y_ext)/2)

40.157466313491653